In [1]:
from pyspark.sql import SparkSession
from delta import *

builder = (
    SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/nil/anaconda3/envs/spark/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nil/.ivy2/cache
The jars for the packages stored in: /home/nil/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7fbe97e0-b966-4421-8ead-415c0b629930;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 207ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

In [2]:
# create sample delta table
DeltaTable.createOrReplace(sparkSession=spark).tableName("dimEmployee").addColumn(
    "empId", "INT"
).addColumn("empName", "String").addColumn("SSN", "String").location(
    "/mnt/sda3/iNeuron/Data_Engineering/PySpark/Delta_Lake/delta_lake_08"
).execute()

24/02/26 20:05:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


24/02/26 20:06:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# insert data
spark.sql(
    "insert into dimEmployee values(100,'Mike','2345671'),(200,'David','5632178'),(300,'Mike','1456782')"
)

DataFrame[]

In [6]:
# view data
spark.sql("select * from dimEmployee").show()

+-----+-------+-------+
|empId|empName|    SSN|
+-----+-------+-------+
|  200|  David|5632178|
|  300|   Mike|1456782|
|  100|   Mike|2345671|
+-----+-------+-------+



pip install cryptography

In [7]:
# generate encryption/decryption key
from cryptography.fernet import Fernet

In [8]:
key = Fernet.generate_key()
f = Fernet(key)

In [9]:
# define UDf to Encrypt Data
def encrypt_data(data, key):
    f = Fernet(key)
    dataDB = bytes(data, "utf-8")
    encrypted_data = f.encrypt(dataDB)
    encrypted_data = str(encrypted_data.decode("ascii"))
    return encrypted_data

In [10]:
# define UDF to Decrypt Data
def decrypt_data(encrypted_data, key):
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data.encode()).decode()
    return decrypted_data

In [11]:
# register UDF's
from pyspark.sql.functions import udf, lit, md5
from pyspark.sql.types import StringType

encryption = udf(encrypt_data, StringType())
decryption = udf(decrypt_data, StringType())

In [14]:
# Encrypt the data
df = spark.table("dimEmployee")
encryptedD = df.withColumn("ssn_encrypted", encryption("SSN", lit(key)))
encryptedD.show(truncate=False)

+-----+-------+-------+----------------------------------------------------------------------------------------------------+
|empId|empName|SSN    |ssn_encrypted                                                                                       |
+-----+-------+-------+----------------------------------------------------------------------------------------------------+
|200  |David  |5632178|gAAAAABl3KizjZ_OAqCMwTiU3TwLTCGOFHUo3EzdLHm_89K631AbORuIK3bL7GdzyXi-TkAHp3gkXYtm3y0B3mneDUGjzVYMGA==|
|300  |Mike   |1456782|gAAAAABl3Ki0DJwQ1d9PdO0J0wLOIpyrmyiypB0jSnnSQOT9ancnk-N-fBAsC5wlbqlmUXcew8PEYabwSH6Cs5C_19V-bXFJ5A==|
|100  |Mike   |2345671|gAAAAABl3Ki0zPFq82PYmW8pkvL8A090uXlfCLuRrqPftiuvNS0xeqA-mgKlE6wOgBCgJMIwn5JJGm7XU-2QVUR732vXtIUmaw==|
+-----+-------+-------+----------------------------------------------------------------------------------------------------+



In [15]:
# Decrypt the data
dcryptedD = encryptedD.withColumn(
    "ssn_decrypted", decryption("ssn_encrypted", lit(key))
)
dcryptedD.show(truncate=False)

+-----+-------+-------+----------------------------------------------------------------------------------------------------+-------------+
|empId|empName|SSN    |ssn_encrypted                                                                                       |ssn_decrypted|
+-----+-------+-------+----------------------------------------------------------------------------------------------------+-------------+
|200  |David  |5632178|gAAAAABl3KkNEZfbLobvk3c0IHJLQb0EZAjuduJ2xjIRwlvjVPJxWNxZP6RWSrIY2eW1H5dTiLBBtkjyji-URQyEfG4gOEinlA==|5632178      |
|300  |Mike   |1456782|gAAAAABl3KkN9vaPdHd8mIYvxr_Rd8L4DjbqcPUfRxLl9tbYGMI5Cn004DnkRfRMDiidVGc3XZthHRo1nRdd8DDqRBXaiqLbxw==|1456782      |
|100  |Mike   |2345671|gAAAAABl3KkNPP_V1hIuVcCawEqcmifGRzThxH5FhloLYBxQd1cs-qNUAlBKMvYfSUz4RocI3AH6apfE7VJXU6gDu0IiO3sulg==|2345671      |
+-----+-------+-------+----------------------------------------------------------------------------------------------------+-------------+

